# Home Match
The following project explores vector databases and prompt engineering to generate and recommend real estate listings to users.

In [1]:
# pip install -r requirements.txt

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import os
import chromadb
import pandas as pd

from pprint import pprint
from functools import partial
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL_NAME = "gpt-3.5-turbo"

# Temperature is very inportant for generating better listings. If its set to zero, all the listings have a similar "vibe".
TEMPERATURE = 0.1

llm = OpenAI(
    model_name=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens = 4096,
    openai_api_key=OPENAI_API_KEY
)

chromadb_client = chromadb.Client()
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
collection = chromadb_client.get_or_create_collection("home-match", embedding_function=embeddings.embed_documents)

/Users/tyler/miniconda3/envs/genai/lib/python3.11/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/tyler/miniconda3/envs/genai/lib/python3.11/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


The following functions are defined in the `utils.py` file

In [4]:
from utils import (
    generate_listing,
    see_collection,
    load_listing_into_db,
    get_listings_from_query,
    get_personalized_descriptions,
    get_personalized_listings
)

# Synthetic Data Generation
LLMs can be use effectivly to generate data. This section shows some experimentation with this functionality.

## Generating Real Estate Listings with an LLM
In this section we will be using a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

In [5]:
NUM_TO_GENERATE = 20

In [6]:
df_listings = pd.DataFrame([generate_listing(i, llm) for i in range(1, NUM_TO_GENERATE+1)])

# Save the listings for Later
df_listings.to_json('listings.json',index=False, orient='records')

In [15]:
df_listings.head(5)

,listing_id,property_type,address,neighborhood,price,bedrooms,bathrooms,description,neighborhood_description,sqft
0,1,townhouse,8780 Cypress St.,Kerrisdale,950000,3,2.5,Welcome to this charming townhouse located in ...,Experience the best of Kerrisdale living in th...,1200
1,2,townhouse,9837 Cypress St.,Kerrisdale,1200000,3,2.5,Welcome to this stunning townhouse located in ...,Experience the charm of Kerrisdale living in t...,1500
2,3,house,5192 Cypress Ave.,Kerrisdale,1200000,4,3.0,Welcome to this charming family home located i...,Experience the best of Kerrisdale living in th...,2500
3,4,house,9453 Cedar St.,Kitsilano,1200000,4,3.0,"Welcome to this stunning 4 bedroom, 3 bathroom...",Experience the best of Vancouver living in Kit...,2000
4,5,condo,9712 Cypress Ave.,Downtown Vancouver,950000,2,2.0,"Welcome to this stunning 2 bedroom, 2 bathroom...",Experience the vibrant energy of Downtown Vanc...,1100


# Semantic Search

In this section I will experiment with using a vector database to perform semantic search.

## Creating a Vector Database and Storing Listings
Lets show the creation of a vector database and successfully storing real estate listing embeddings within it. The database should effectively store and organize the embeddings generated from the LLM-created listings.

In [7]:
df_listings = pd.read_json('listings.json')

In [8]:
# Create a partial function with the additional input
partial_func = partial(load_listing_into_db, collection=collection)

# Apply the partial function to each row in the DataFrame
df_listings.apply(partial_func, axis=1)

see_collection(collection)

Collection: home-match
Number of Items: 20

Items in Collection:
ids
embeddings
metadatas
documents


## Semantic Search of Listings Based on Buyer Preferences

Now lets show some functionality where listings are semantically searched based on given buyer preferences. The search should return listings that closely match the input preferences.

In [9]:
query1 = "I love Open concept"
pprint(get_listings_from_query(user_query=query1, collection=collection))

[{'address': '9712 Cypress Ave.',
  'bathrooms': 2.0,
  'bedrooms': 2,
  'description': 'Welcome to this stunning 2 bedroom, 2 bathroom condo located '
                 'in the heart of Downtown Vancouver. This modern condo '
                 'features an open concept layout with floor-to-ceiling '
                 'windows, allowing for plenty of natural light to flow '
                 'through the space. The kitchen is equipped with stainless '
                 'steel appliances, quartz countertops, and a large island '
                 'perfect for entertaining. The master bedroom boasts a '
                 'walk-in closet and ensuite bathroom with a luxurious soaking '
                 'tub. Enjoy the convenience of in-suite laundry and a private '
                 'balcony with city views. This condo is perfect for urban '
                 'living at its finest.\n'
                 'Experience the vibrant energy of Downtown Vancouver living '
                 'at its best in thi

In [10]:
query2 = "Close to the beach"
pprint(get_listings_from_query(user_query=query2, collection=collection))

[{'address': '593 Oceanview Ave.',
  'bathrooms': 2.5,
  'bedrooms': 3,
  'description': 'Welcome to this stunning townhouse located in the heart of '
                 'Kitsilano. This beautifully designed home features an open '
                 'concept layout with high ceilings, hardwood floors, and '
                 'large windows that fill the space with natural light. The '
                 'modern kitchen is equipped with stainless steel appliances, '
                 'quartz countertops, and a large island perfect for '
                 'entertaining. Upstairs, you will find three spacious '
                 'bedrooms including a master suite with a walk-in closet and '
                 'ensuite bathroom. Enjoy the outdoors on your private patio '
                 'or take a short walk to the beach and enjoy all that '
                 "Kitsilano has to offer. Don't miss out on this opportunity "
                 "to own a piece of paradise in one of Vancouver's most "
       

In [11]:
query3 = "Walking distance to nice restaurants"
pprint(get_listings_from_query(user_query=query3, collection=collection))

[{'address': '9412 Cedar St.',
  'bathrooms': 2.5,
  'bedrooms': 3,
  'description': 'Welcome to this stunning townhouse located in the heart of '
                 'Mount Pleasant. This beautifully designed home features 3 '
                 'spacious bedrooms, 2.5 bathrooms, and a modern open concept '
                 'layout. The main floor boasts a gourmet kitchen with '
                 'stainless steel appliances, quartz countertops, and a large '
                 'island perfect for entertaining. The living room is bright '
                 'and airy with large windows that let in plenty of natural '
                 "light. Upstairs, you'll find the luxurious master suite with "
                 'a walk-in closet and ensuite bathroom. The two additional '
                 'bedrooms are perfect for guests or a home office. This '
                 'townhouse also includes a private patio, perfect for '
                 'enjoying your morning coffee or hosting a summer BBQ. '
    

In [12]:
# This one is cool. The word "bougie" is slang and not present in the dataset.
query3 = "Show me the bougie options"
pprint(get_listings_from_query(user_query=query3, collection=collection))

[{'address': '9099 Oceanview Ave.',
  'bathrooms': 2.0,
  'bedrooms': 2,
  'description': 'Welcome to this luxurious condo located in the heart of '
                 'Downtown Vancouver. This stunning unit offers breathtaking '
                 'views of the city skyline and waterfront. The open concept '
                 'layout is perfect for entertaining, with a modern kitchen '
                 'featuring high-end appliances and a spacious living area. '
                 'The master bedroom boasts a walk-in closet and en-suite '
                 'bathroom with a deep soaking tub. Enjoy the convenience of '
                 'being steps away from top restaurants, shopping, and '
                 'entertainment. Building amenities include a fitness center, '
                 "pool, and 24-hour concierge service. Don't miss out on this "
                 'rare opportunity to own a piece of urban paradise.\n'
                 'Experience the vibrant energy of Downtown Vancouver in this

# Augmented Response Generation

## Logic for Searching and Augmenting Listing Descriptions
I will now demonstrate a logical flow where buyer preferences are used to search and then augment the description of real estate listings. The augmentation should personalize the listing without changing factual information.

Lets utilize an LLM to generate personalized descriptions for the real estate listings based on buyer preferences. The descriptions should be unique, appealing, and tailored to the preferences provided.

Lets pretend like we have set up a flow that records the following responses from the human

In [13]:
questions = [
    "What is your name?",
    "How big do you want your house to be?",
    "What is the most important thing about your new house",  
    "Which transportation options are important to you?",   
]
answers = [
    "Doug McDude",
    "A comfortable two-bedroom house with a spacious kitchen and a cozy living room.",
    "That its bougie.",
    "Its important that it is close to the outdoors",
]

Now we can get personalized responses.

In [14]:
get_personalized_listings(questions, answers, llm, collection)

[{'listing_id': '5',
  'description': "Welcome to this stunning 2 bedroom, 2 bathroom condo located in the heart of Downtown Vancouver, Doug McDude. This modern condo is the perfect size for you, with a comfortable two-bedroom layout that includes a spacious kitchen and cozy living room. The open concept design and floor-to-ceiling windows allow for plenty of natural light to flow through the space, creating a warm and inviting atmosphere. \n\nThe kitchen is equipped with stainless steel appliances, quartz countertops, and a large island perfect for entertaining - just what you were looking for in a new house. The master bedroom boasts a walk-in closet and ensuite bathroom with a luxurious soaking tub, adding a touch of luxury to your everyday life. \n\nYou mentioned that it's important for your new house to be bougie, and this condo certainly fits the bill with its high-end finishes and modern design. Additionally, you value being close to the outdoors, and this condo offers a private

We can see the descriptions are now personalized to our user `Doug McDude` and his perferences answered by the questions.